# 04a - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

This notebook shows training a model directly within the runtime of the notebook environment.  Then the model is saved and moved to GCS for deployment to a Vertex AI Endpoint for online predictions.  The model training is done with [Tensorflow](https://www.tensorflow.org/), specifically [Keras](https://keras.io/), and was designed to show a neural network approach to logistic regression.  The training data batches are read from BigQuery using [Tensorflow I/O](https://www.tensorflow.org/io).

**Prerequisites:**

-  01 - BigQuery - Table Data Source

**Overview:**

-  Use Python Client for BigQuery
   -  Read the tables schema from BigQuery INFORMATION_SCHEMA
   -  Prepare the feature information for Tensorflow
-  Define a function that remaps the input data into features and target variables where target is one-hot encoded (classification model with 10 classes)
-  Set Tensorflow I/O read session
-  Demonstrate reading a single batch
-  Train a Tensorflow model
   -  Define the model layers
   -  Compile the model
   -  Fit the model
   -  Evaluate the model (loss, accuracy)
   -  Create prediction with the model
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict
-  Online Predictions with:
   -  REST call
   -  gcloud CLI

**Resources:**

-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)

**Related Training:**

-  todo

---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/slide_17.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/slide_18.png">

---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'fraud'
NOTEBOOK = '04a'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [2]:
from google.cloud import bigquery

from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf

from google.cloud import aiplatform
from datetime import datetime

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

2021-10-11 15:22:50.516226: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA


clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

environment:

In [5]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get The Schema of The Training Data

In [6]:
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

In [7]:
schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,statmike-demo,fraud,fraud_prepped,Time,1,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
1,statmike-demo,fraud,fraud_prepped,V1,2,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
2,statmike-demo,fraud,fraud_prepped,V2,3,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
3,statmike-demo,fraud,fraud_prepped,V3,4,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
4,statmike-demo,fraud,fraud_prepped,V4,5,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
5,statmike-demo,fraud,fraud_prepped,V5,6,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
6,statmike-demo,fraud,fraud_prepped,V6,7,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
7,statmike-demo,fraud,fraud_prepped,V7,8,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
8,statmike-demo,fraud,fraud_prepped,V8,9,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
9,statmike-demo,fraud,fraud_prepped,V9,10,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN


## Number of Classes for the VAR_TARGET?

In [8]:
nclasses = bigquery.query(query = f'SELECT DISTINCT {VAR_TARGET} FROM {DATANAME}.{DATANAME}_prepped WHERE {VAR_TARGET} is not null').to_dataframe()

In [9]:
nclasses

,Class
0,0
1,1


In [10]:
nclasses = nclasses.shape[0]
nclasses

2

## Prepare Inputs For Tensorflow Training

Use the the table schema to prepare the TensorFlow Model:
- Omit unused columns
- Create `feature_columns` for the model
- Define the `dtypes` for TensorFlow

In [11]:
OMIT = VAR_OMIT.split() + ['splits']

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != VAR_TARGET:
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

# all the columns in this data source are either float64 or int64
output_types = schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

Define a function that remaps the input data for TensorFlow into:
- features
- `target` - and one_hot encoded for multi-class classification

In [12]:
def transTable(row_dict):
    target=row_dict.pop(VAR_TARGET)
    target = tf.one_hot(tf.cast(target,tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

## Use Tensorflow I/O to Read Batches from BigQuery

Setup TensorFlow_IO client > session > table + table.map
- https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient

In [13]:
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

In [14]:
train = bq_reader('TRAIN').parallel_read_rows().map(transTable).shuffle(BATCH_SIZE*10).batch(BATCH_SIZE)
validate = bq_reader('VALIDATE').parallel_read_rows().map(transTable).batch(BATCH_SIZE)
test = bq_reader('TEST').parallel_read_rows().map(transTable).batch(BATCH_SIZE)

2021-10-11 15:23:33.489139: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2299995000 Hz
2021-10-11 15:23:33.489942: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ac9e99d930 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-11 15:23:33.489971: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-11 15:23:33.491933: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Review a single batch of the train data:

In [15]:
for a, b in train.take(1):
    columns=list(a.keys())
    print('features:\n',columns)
    print('\ntarget:\n',b[0:10])

features:
 ['Amount', 'Time', 'V1', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V2', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9']

target:
 tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]], shape=(10, 2), dtype=float32)


---
## Train the Model In The Notebook

Define the Model:
Adding metrics to the model:
- Specifying 'accuracy' triggers conversion to tf.keras.metrics.CategoricalAccuracy due to the loss function being tf.keras.losses.CategoricalCrossentropy
- If the target variable is highly imbalanced then it is good to consider the area under the precision-recall curve.  This is requested with tf.keras.metrics.AUC(curve='PR')

In [16]:
# Simple Logistic Regression
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)

layers = tf.keras.layers.BatchNormalization()(feature_layer_outputs)
layers = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax)(layers)

model = tf.keras.Model(
    inputs = [v for v in feature_layer_inputs.values()],
    outputs = layers
)
opt = tf.keras.optimizers.SGD() #SGD or Adam
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy', tf.keras.metrics.AUC(curve='PR')]
)

In [17]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Amount (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
Time (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
V1 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
V10 (InputLayer)                [(None, 1)]          0                                            
_______________________________________________________________________________________

Fit the Model:

In [18]:
history = model.fit(train, epochs = EPOCHS, validation_data = validate)

Epoch 1/10
2279/2279 [==============================] - 76s 34ms/step - loss: 0.0883 - accuracy: 0.9830 - auc: 0.9957 - val_loss: 0.0409 - val_accuracy: 0.9975 - val_auc: 0.9969
Epoch 2/10
2279/2279 [==============================] - 78s 34ms/step - loss: 0.0205 - accuracy: 0.9981 - auc: 0.9983 - val_loss: 0.0179 - val_accuracy: 0.9982 - val_auc: 0.9992
Epoch 3/10
2279/2279 [==============================] - 78s 34ms/step - loss: 0.0102 - accuracy: 0.9985 - auc: 0.9997 - val_loss: 0.0133 - val_accuracy: 0.9985 - val_auc: 0.9993
Epoch 4/10
2279/2279 [==============================] - 78s 34ms/step - loss: 0.0073 - accuracy: 0.9989 - auc: 0.9997 - val_loss: 0.0118 - val_accuracy: 0.9986 - val_auc: 0.9994
Epoch 5/10
2279/2279 [==============================] - 77s 34ms/step - loss: 0.0063 - accuracy: 0.9990 - auc: 0.9997 - val_loss: 0.0111 - val_accuracy: 0.9987 - val_auc: 0.9994
Epoch 6/10
2279/2279 [==============================] - 79s 35ms/step - loss: 0.0057 - accuracy: 0.9990 - auc:

In [19]:
history.history['loss'][-1]

0.0048231808468699455

Evaluate the model with the test data:

In [21]:
loss, accuracy, auc = model.evaluate(test)

285/285 [==============================] - 5s 18ms/step - loss: 0.0086 - accuracy: 0.9990 - auc: 0.9995


In [22]:
loss, accuracy, auc = model.evaluate(validate)

286/286 [==============================] - 5s 19ms/step - loss: 0.0097 - accuracy: 0.9989 - auc: 0.9994


In [23]:
loss, accuracy, auc = model.evaluate(train)

2279/2279 [==============================] - 76s 33ms/step - loss: 0.0073 - accuracy: 0.9989 - auc: 0.9996


Create Prediction from a batch of the test data and review first row:

In [24]:
model.predict(test.take(1))[0]

array([9.99889255e-01, 1.10731475e-04], dtype=float32)

---
## Serving

### Save The Model

In [25]:
model.save(URI)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2021-10-11 16:16:06.745482: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://statmike-demo/fraud/models/04a/assets
INFO:tensorflow:Assets written to: gs://statmike-demo/fraud/models/04a/assets


### Upload The Model

In [26]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/509126313368/locations/us-central1/models/3450029993449488384/operations/7835348695389306880
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/509126313368/locations/us-central1/models/3450029993449488384
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/509126313368/locations/us-central1/models/3450029993449488384')


In [27]:
model.display_name

'04a_fraud_20211011152251'

### Create An Endpoint

In [28]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/509126313368/locations/us-central1/endpoints/9090498255661301760/operations/6362108667285733376
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/509126313368/locations/us-central1/endpoints/9090498255661301760
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/509126313368/locations/us-central1/endpoints/9090498255661301760')


In [29]:
endpoint.display_name

'04a_fraud_20211011152251'

### Deploy Model To Endpoint

In [30]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/509126313368/locations/us-central1/models/3450029993449488384 to Endpoint : projects/509126313368/locations/us-central1/endpoints/9090498255661301760
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/509126313368/locations/us-central1/endpoints/9090498255661301760/operations/1237012291338108928
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/509126313368/locations/us-central1/endpoints/9090498255661301760


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [31]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [32]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,42525,-0.646143,0.736469,0.942179,1.475040,2.130779,4.972248,-0.940231,0.500023,-0.272361,...,-0.224510,1.039882,0.027130,0.262897,0.232499,0.146983,0.0,0,3e132b2b-63c1-4aa6-9e40-f8f6788a12c7,TEST
1,56319,0.792079,0.436581,1.358800,2.798437,-0.769028,0.087739,-0.470308,0.298705,0.300321,...,0.340340,0.312103,-1.132621,-0.162690,0.008280,-0.111146,0.0,0,bb5bdb3c-a1c0-45b7-a6db-5580f95f1e29,TEST
2,142598,2.210514,0.141203,-2.841177,0.254088,1.197321,-1.018585,0.871035,-0.467864,0.079799,...,-0.220533,0.097614,0.758281,0.737860,-0.137577,-0.096846,0.0,0,eb4a161a-8e36-4aa0-9555-391249d2374b,TEST
3,39386,-1.092735,0.210722,2.937391,2.481408,0.379506,1.302458,-0.675156,0.711658,-0.471588,...,-0.341725,-0.264885,0.341552,0.402259,0.100453,0.077091,0.0,0,758afcfb-757d-430c-a4a0-d26f05f7cd24,TEST


In [33]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [34]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [35]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999889255, 0.000110731475]], deployed_model_id='1797719103599804416', explanations=None)

In [36]:
prediction.predictions[0]

[0.999889255, 0.000110731475]

In [37]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [38]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [39]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999889255,
      0.000110731475
    ]
  ],
  "deployedModelId": "1797719103599804416",
  "model": "projects/509126313368/locations/us-central1/models/3450029993449488384",
  "modelDisplayName": "04a_fraud_20211011152251"
}


### Get Predictions: gcloud (CLI)

In [40]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999889255, 0.000110731475]]


---
## Remove Resources
see notebook "XX - Cleanup"